## Identify possible boulder-quakes among Geophone 3 and 4 events

### Import libraries

In [1]:
import pandas as pd
from obspy import read,UTCDateTime
from datetime import datetime, timedelta
import numpy as np
import os
import glob
import sys
import matplotlib.pyplot as plt
from PIL import Image

# Import functions
fxndir = '../functions/'
sys.path.insert(0,fxndir)
from moon2data import *

### Load catalogs of Geophone 3 and 4 events

In [2]:
# Geophone 3 events
mqdir = '../catalogs/final_catalogs/geo3_geo4_events/'
cat1_geo3 = pd.read_csv(mqdir + 'Geophone3_events_catalog_HQ_final.csv')
cat2_geo3 = pd.read_csv(mqdir + 'Geophone3_events_catalog_HQ_avg_event_stats.csv')

# Geophone 4 events
cat1_geo4 = pd.read_csv(mqdir + 'Geophone4_events_catalog_HQ_final.csv')
cat2_geo4 = pd.read_csv(mqdir + 'Geophone4_events_catalog_HQ_avg_event_stats.csv')

### Inputs to obtain waveforms

In [3]:
parentdir = '/data/ytamama/Apollo17/LSPE_data/sac_volts_ds/'
minfreq = 3
maxfreq = 35
befwin = 2
aftwin = 12

### Identify possible Geophone Rock and R2 events based on relative amplitudes between geophones

In [4]:
possible_sources = []
for r in np.arange(0,len(cat2_geo3)):
    
    row = cat2_geo3.iloc[r]
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    
    # Traces at each geophone
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    
    # Peak ground velocity at each geophone
    pgv_geo1 = np.max(np.abs(trdata1))
    pgv_geo2 = np.max(np.abs(trdata2))
    pgv_geo3 = np.max(np.abs(trdata3))
    pgv_geo4 = np.max(np.abs(trdata4))
    
    # Mean absolute amplitude at each geophone
    mean_geo1 = np.mean(np.abs(trdata1))
    mean_geo2 = np.mean(np.abs(trdata2))
    mean_geo3 = np.mean(np.abs(trdata3))
    mean_geo4 = np.mean(np.abs(trdata4))
    
    # Classify by possible source
    df = pd.DataFrame(data = {'geophones':[1, 2, 3, 4],
                                'PGV':np.array([pgv_geo1, pgv_geo2, pgv_geo3, pgv_geo4]),
                                'mean':np.array([mean_geo1, mean_geo2, mean_geo3, mean_geo4])})
    # PGV
    df_pgv = df.sort_values(by=['PGV'],ascending=False)
    geophones_pgv = df_pgv.geophones.tolist()
    if (geophones_pgv[0] == 3) & (geophones_pgv[1] == 4) & (geophones_pgv[-1] == 1):
        source_pgv = 'geophone_rock'
    elif (geophones_pgv[0] == 3) & (geophones_pgv[1] == 1) & (geophones_pgv[-1] == 4):
        source_pgv = 'R2'
    else:
        source_pgv = 'unclear'
    
    # Mean amplitude
    df_mean = df.sort_values(by=['mean'],ascending=False)
    geophones_mean = df_mean.geophones.tolist()
    if (geophones_mean[0] == 3) & (geophones_mean[1] == 4) & (geophones_mean[-1] == 1):
        source_mean = 'geophone_rock'
    elif (geophones_mean[0] == 3) & (geophones_mean[1] == 1) & (geophones_mean[-1] == 4):
        source_mean = 'R2'
    else:
        source_mean = 'unclear'
        
    # Classify by possible source
    if (source_pgv == 'geophone_rock') | (source_mean == 'geophone_rock'):
        possible_sources.append('geophone_rock')
    elif (source_pgv == 'R2') | (source_mean == 'R2'):
        possible_sources.append('R2')
    else:
        possible_sources.append('unclear')
        
# Append possible source information to catalogs
cat2_geo3['possible_source'] = possible_sources

In [5]:
# Incorporate full families
possible_sources_new = possible_sources.copy()
evids_ref_georock = np.unique(cat2_geo3.loc[cat2_geo3.possible_source == 'geophone_rock'].evid_ref.tolist())
evids_ref_R2 = np.unique(cat2_geo3.loc[cat2_geo3.possible_source == 'R2'].evid_ref.tolist())
for r in np.arange(0,len(cat2_geo3)):
    row = cat2_geo3.iloc[r]
    evid_ref = row.evid_ref
    if evid_ref in evids_ref_georock:
        possible_sources_new[r] = 'geophone_rock'
    elif evid_ref in evids_ref_R2:
        possible_sources_new[r] = 'R2'
        
# Catalog 2
cat2_geo3['possible_source'] = possible_sources_new

# Catalog 1
possible_sources1 = []
for r in np.arange(0,len(cat1_geo3)):
    row = cat1_geo3.iloc[r]
    evid = row.evid
    row2 = cat2_geo3.loc[cat2_geo3.evid == evid].iloc[0]
    possible_sources1.append(row2.possible_source) 
cat1_geo3['possible_source'] = possible_sources1

In [6]:
len(cat2_geo3.loc[cat2_geo3.possible_source == 'geophone_rock'])

31

In [7]:
len(cat2_geo3.loc[cat2_geo3.possible_source == 'R2'])

9

In [8]:
# Save catalog
cat1_geo3.to_csv(mqdir + 'Geophone3_events_catalog_HQ_final.csv',index=False)
cat2_geo3.to_csv(mqdir + 'Geophone3_events_catalog_HQ_avg_event_stats.csv',index=False)

### Plot possible boulderquakes as a sanity check

#### Geophone Rock

In [9]:
savedir = './boulderquake_waveforms/'
pngnames = []
cat2_geo3 = cat2_geo3.sort_values(by=['evid_ref'],ignore_index=True)
evids_georock = cat2_geo3.loc[cat2_geo3.possible_source == 'geophone_rock'].evid.tolist()

# Iteratively plot events
for evid in evids_georock:
    row = cat2_geo3.loc[cat2_geo3.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    isol_or_rpt = row.isol_or_rpt
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0',alpha=0.75)
    ax.plot(trtimes2,trdata2_norm+4, color='C1',alpha=0.75)
    ax.plot(trtimes3,trdata3_norm+2, color='C2',alpha=0.75)
    ax.plot(trtimes4,trdata4_norm, color='C3',alpha=0.75)
    ax.set_title(f'EVID {evid} (REF={evid_ref}, {isol_or_rpt})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'REF' + evid_ref + '_EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'REF' + evid_ref + '_EVID' + evid + '_waveforms.png')
    plt.close()

In [10]:
# Combine figures into one PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = savedir + 'possible_geophonerock_events.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

#### R2

In [11]:
pngnames = []
evids_R2 = cat2_geo3.loc[cat2_geo3.possible_source == 'R2'].evid.tolist()

# Iteratively plot events
for evid in evids_R2:
    row = cat2_geo3.loc[cat2_geo3.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    isol_or_rpt = row.isol_or_rpt
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0',alpha=0.75)
    ax.plot(trtimes2,trdata2_norm+4, color='C1',alpha=0.75)
    ax.plot(trtimes3,trdata3_norm+2, color='C2',alpha=0.75)
    ax.plot(trtimes4,trdata4_norm, color='C3',alpha=0.75)
    ax.set_title(f'EVID {evid} (REF={evid_ref}, {isol_or_rpt})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'REF' + evid_ref + '_EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'REF' + evid_ref + '_EVID' + evid + '_waveforms.png')
    plt.close()

In [12]:
# Combine figures into one PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = savedir + 'possible_R2_events.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

### Identify possible Geophone 4 Rock events based on relative amplitudes between geophones

In [13]:
possible_sources = []
for r in np.arange(0,len(cat2_geo4)):
    row = cat2_geo4.iloc[r]
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    
    # Traces at each geophone
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    
    # Peak ground velocity at each geophone
    pgv_geo1 = np.max(np.abs(trdata1))
    pgv_geo2 = np.max(np.abs(trdata2))
    pgv_geo3 = np.max(np.abs(trdata3))
    pgv_geo4 = np.max(np.abs(trdata4))
    
    # Mean absolute amplitude at each geophone
    mean_geo1 = np.mean(np.abs(trdata1))
    mean_geo2 = np.mean(np.abs(trdata2))
    mean_geo3 = np.mean(np.abs(trdata3))
    mean_geo4 = np.mean(np.abs(trdata4))
    
    # Classify by possible source
    df = pd.DataFrame(data = {'geophones':[1, 2, 3, 4],
                                'PGV':np.array([pgv_geo1, pgv_geo2, pgv_geo3, pgv_geo4]),
                                'mean':np.array([mean_geo1, mean_geo2, mean_geo3, mean_geo4])})
    # PGV
    df_pgv = df.sort_values(by=['PGV'],ascending=False)
    geophones_pgv = df_pgv.geophones.tolist()
    if (geophones_pgv[0] == 4) & (geophones_pgv[1] == 3) & (geophones_pgv[-1] == 2):
        source_pgv = 'GEO4_rock1'
    elif (geophones_pgv[0] == 4) & (geophones_pgv[1] == 3) & (geophones_pgv[-1] == 1):
        source_pgv = 'GEO4_rock2or3'
    else:
        source_pgv = 'unclear'
        
    # Mean amplitude
    df_mean = df.sort_values(by=['mean'],ascending=False)
    geophones_mean = df_mean.geophones.tolist()
    if (geophones_mean[0] == 4) & (geophones_mean[1] == 3) & (geophones_mean[-1] == 2):
        source_mean = 'GEO4_rock1'
    elif (geophones_mean[0] == 4) & (geophones_mean[1] == 3) & (geophones_mean[-1] == 1):
        source_mean = 'GEO4_rock2or3'
    else:
        source_mean = 'unclear'

    # Classify by possible source
    if (source_pgv == 'GEO4_rock1') | (source_mean == 'GEO4_rock1'):
        possible_sources.append('GEO4_rock1')
    elif (source_pgv == 'GEO4_rock2or3') | (source_mean == 'GEO4_rock2or3'):
        possible_sources.append('GEO4_rock2or3')
    else:
        possible_sources.append('unclear')

# Append possible source information to catalogs
cat2_geo4['possible_source'] = possible_sources

In [14]:
# Incorporate full families
possible_sources_new = possible_sources.copy()
evids_ref_rock1 = np.unique(cat2_geo4.loc[cat2_geo4.possible_source == 'GEO4_rock1'].evid_ref.tolist())
evids_ref_rock23 = np.unique(cat2_geo4.loc[cat2_geo4.possible_source == 'GEO4_rock2or3'].evid_ref.tolist())
for r in np.arange(0,len(cat2_geo4)):
    row = cat2_geo4.iloc[r]
    evid_ref = row.evid_ref
    if evid_ref in evids_ref_georock:
        possible_sources_new[r] = 'GEO4_rock1'
    elif evid_ref in evids_ref_R2:
        possible_sources_new[r] = 'GEO4_rock2or3'
        
# Catalog 2
cat2_geo4['possible_source'] = possible_sources_new

# Catalog 1
possible_sources1 = []
for r in np.arange(0,len(cat1_geo4)):
    row = cat1_geo4.iloc[r]
    evid = row.evid
    row2 = cat2_geo4.loc[cat2_geo4.evid == evid].iloc[0]
    possible_sources1.append(row2.possible_source)
cat1_geo4['possible_source'] = possible_sources1

In [15]:
len(cat2_geo4.loc[cat2_geo4.possible_source == 'GEO4_rock1'])

12

In [16]:
len(cat2_geo4.loc[cat2_geo4.possible_source == 'GEO4_rock2or3'])

7

In [17]:
# Save catalog
cat1_geo4.to_csv(mqdir + 'Geophone4_events_catalog_HQ_final.csv',index=False)
cat2_geo4.to_csv(mqdir + 'Geophone4_events_catalog_HQ_avg_event_stats.csv',index=False)

### Plot possible boulderquakes as a sanity check

#### Geophone 4 Rock 1

In [18]:
savedir = './boulderquake_waveforms/'
pngnames = []
evids_rock1 = cat2_geo4.loc[cat2_geo4.possible_source == 'GEO4_rock1'].evid.tolist()

# Iteratively plot events
for evid in evids_rock1:
    row = cat2_geo4.loc[cat2_geo4.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    isol_or_rpt = row.isol_or_rpt
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0',alpha=0.75)
    ax.plot(trtimes2,trdata2_norm+4, color='C1',alpha=0.75)
    ax.plot(trtimes3,trdata3_norm+2, color='C2',alpha=0.75)
    ax.plot(trtimes4,trdata4_norm, color='C3',alpha=0.75)
    ax.set_title(f'EVID {evid} (REF={evid_ref}, {isol_or_rpt})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [19]:
# Combine figures into one PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = savedir + 'possible_GEO4Rock1_events.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

#### Geophone 4 Rock 2

In [20]:
pngnames = []
evids_rock23 = cat2_geo4.loc[cat2_geo4.possible_source == 'GEO4_rock2or3'].evid.tolist()

# Iteratively plot events
for evid in evids_rock23:
    row = cat2_geo4.loc[cat2_geo4.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    isol_or_rpt = row.isol_or_rpt
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0',alpha=0.75)
    ax.plot(trtimes2,trdata2_norm+4, color='C1',alpha=0.75)
    ax.plot(trtimes3,trdata3_norm+2, color='C2',alpha=0.75)
    ax.plot(trtimes4,trdata4_norm, color='C3',alpha=0.75)
    ax.set_title(f'EVID {evid} (REF={evid_ref}, {isol_or_rpt})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [21]:
# Combine figures into one PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = savedir + 'possible_GEO4Rock2or3_events.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)